In [1]:
import ast
import json
import os
import pickle
from pathlib import Path

import openai
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# トークン数を記録する変数を初期化
total_input_tokens = 0
total_output_tokens = 0

In [3]:
# 使用するモデルの設定
MODEL_NAME = "gpt-3.5-turbo"

# .envファイルのロード
load_dotenv(dotenv_path=Path("./data/.env"))

# OpenAI APIキーの設定
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

# モデルのコスト情報のロード
with open("./data/model_costs.json", "r") as f:
    model_costs = json.load(f)

# ラベルのロード
with open("./data/labels.json", "r") as f:
    labels = json.load(f)["labels"]

# 元データのデータフレームのロード
with open("data/df_batches.pkl", "rb") as f:
    df_batches = pickle.load(f)

# 元データの辞書のロード
with open("data/original_data_dict.pkl", "rb") as f:
    original_data_dict = pickle.load(f)

In [4]:
def classify_and_evaluate(texts_dict: dict[str], labels: list[str], model: str) -> dict:
    """
    文章を指定されたラベルに分類し、満足か不満かを判断する関数

    Parameters
    ----------
    texts_dict : dict[str]
        分類する文章の辞書
    labels : list[str]
        分類するラベル
    model : str
        使用するモデル

    Returns
    -------
    dict
        分類結果と満足度の判定結果
    """

    global total_input_tokens, total_output_tokens

    try:
        # OpenAI APIリクエストの作成
        client = OpenAI()
        response = client.chat.completions.create(
            model=model,
            messages=[
                # {
                #     "role": "system",
                #     "content": "あなたは、テキストを事前定義されたラベルに分類し、満足度を評価するアシスタントです。",
                # },
                {
                    "role": "user",
                    "content": f"与えられた文章を次のようなラベルの1つに分類してください: {labels}。"
                    + "\n次に、その文章が満足か不満のどちらであるかを判断してください。\n出力の形式は次のような辞書型にしてください: {text_id : {'label': 'ラベル名', 'sentiment': '満足' または '不満'}, text_id: ...}。"
                    + f"\n文章: {texts_dict}",
                },
            ],
            temperature=1.0,
            # max_tokens=100,
        )

        # レスポンスからトークン数を取得
        input_tokens = response.usage.prompt_tokens
        output_tokens = response.usage.completion_tokens
        total_input_tokens += input_tokens
        total_output_tokens += output_tokens

        # 結果の抽出
        result_text = response.choices[0].message.content
        # 文字列を辞書型に変換
        result_dict = ast.literal_eval(result_text)

        return {
            "result_dict": result_dict,
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
        }

    except Exception as e:
        return {"error": str(e)}


# def evaluate_result(result_dict: dict[str], labels: list[str]) -> dict:


def calculate_cost(input_tokens: int, output_tokens: int, model_name: str) -> float:
    """
    使用したトークン数に基づいてコストを計算する関数

    Parameters
    ----------
    input_tokens : int
        入力トークン数
    output_tokens : int
        出力トークン数
    model_name : str
        使用したモデルの名前

    Returns
    -------
    float
        トークン数に基づいて計算されたコスト(単位: USD)
    """

    input_cost = input_tokens / 1e6 * model_costs[model_name]["input"]
    output_cost = output_tokens / 1e6 * model_costs[model_name]["output"]
    return input_cost + output_cost

In [5]:
# len(df_batches)

In [6]:
result_dict = {}

for df_batch in df_batches:
    # インデックスをキー、文章を値とする辞書に変換
    texts_dict = df_batch["文章"].to_dict()

    # 分類と評価を実行
    result = classify_and_evaluate(texts_dict, labels, MODEL_NAME)
    result_dict.update(result["result_dict"])

In [7]:
len(result_dict)

683

In [8]:
result_dict

{4: {'label': '移動・交通', 'sentiment': '満足'},
 8: {'label': '公共空間', 'sentiment': '満足'},
 12: {'label': '移動・交通', 'sentiment': '満足'},
 13: {'label': '買物・飲食', 'sentiment': '満足'},
 14: {'label': '遊び・娯楽', 'sentiment': '不満'},
 15: {'label': '医療・福祉', 'sentiment': '不満'},
 16: {'label': '公共空間', 'sentiment': '満足'},
 17: {'label': '住宅環境', 'sentiment': '満足'},
 18: {'label': '移動・交通', 'sentiment': '不満'},
 19: {'label': 'デジタル生活', 'sentiment': '不満'},
 31: {'label': '買物・飲食', 'sentiment': '不満'},
 52: {'label': '医療・福祉', 'sentiment': '満足'},
 53: {'label': '移動・交通', 'sentiment': '満足'},
 54: {'label': '自然災害', 'sentiment': '不満'},
 55: {'label': '公共空間', 'sentiment': '不満'},
 57: {'label': '地域行政', 'sentiment': '満足'},
 72: {'label': '自然災害', 'sentiment': '満足'},
 74: {'label': '初等・中等教育', 'sentiment': '不満'},
 75: {'label': '医療・福祉', 'sentiment': '不満'},
 80: {'label': '子育て', 'sentiment': '満足'},
 81: {'label': '自然災害', 'sentiment': '満足'},
 82: {'label': '移動・交通', 'sentiment': '不満'},
 83: {'label': '地域行政', 'sentiment': '不満'}

In [9]:
df_compare = pd.DataFrame(
    {
        "original": {
            key: {
                inner_key: inner_val
                for inner_key, inner_val in val.items()
                if inner_key != "text"
            }
            for key, val in original_data_dict.items()
        },
        "predicted": result_dict,
    }
)

# text列を追加
df_compare["text"] = {key: val["text"] for key, val in original_data_dict.items()}

# NaNを削除
df_compare = df_compare.dropna()

# 一致しているか
df_compare["is_match_all"] = df_compare["original"] == df_compare["predicted"]

In [12]:
df_compare

,original,predicted,text,is_match_all
4,"{'label': '移動・交通', 'sentiment': '満足'}","{'label': '移動・交通', 'sentiment': '満足'}",鉄道路線が充実している。どこにいくにも便利。,True
8,"{'label': '自然景観', 'sentiment': '満足'}","{'label': '公共空間', 'sentiment': '満足'}",海も近く県立公園もあり、リラックスできるので。,False
12,"{'label': '移動・交通', 'sentiment': '満足'}","{'label': '移動・交通', 'sentiment': '満足'}",今居住している武蔵小杉はJR横須賀線、南武線、東急東横線、目黒線と選択の余地がたくさんあり、...,True
13,"{'label': '買物・飲食', 'sentiment': '満足'}","{'label': '買物・飲食', 'sentiment': '満足'}",スーパーマーケットもたくさんあり、選択の余地が広い,True
14,"{'label': '遊び・娯楽', 'sentiment': '不満'}","{'label': '遊び・娯楽', 'sentiment': '不満'}",かなり発展した街なのに映画館がない。ライブハウスがない。,True
...,...,...,...,...
1941,"{'label': 'デジタル生活', 'sentiment': '不満'}","{'label': '地域行政', 'sentiment': '不満'}",行政関係の申請や交付をするときに、市役所まで出向いて行う必要があり、デジタル行政関係が遅れて...,False
1942,"{'label': '医療・福祉', 'sentiment': '不満'}","{'label': '医療・福祉', 'sentiment': '不満'}",高齢者福祉・支援や障害者支援などを包括的に対応できる具体的な体制や組織が不十分であると感じている。,True
1945,"{'label': '医療・福祉', 'sentiment': '不満'}","{'label': '医療・福祉', 'sentiment': '不満'}",南部地区に総合病院がないので設置してほしい。,True
1946,"{'label': '買物・飲食', 'sentiment': '不満'}","{'label': '買物・飲食', 'sentiment': '不満'}",藤沢駅周辺のショッピング環境を更に改善してほしい。,True


In [11]:
with open("data/df_compare.pkl", "wb") as f:
    pickle.dump(df_compare, f)

In [12]:
# 正解率
accuracy = df_compare["is_match"].sum() / len(df_compare)
accuracy

0.6730205278592375

In [13]:
# predicted_dict
# original_data_dict

In [14]:
cost = calculate_cost(total_input_tokens, total_output_tokens, MODEL_NAME)

print(f"Total input tokens: {total_input_tokens}")
print(f"Total output tokens: {total_output_tokens}")
print(f"Total cost: ${cost}")

Total input tokens: 38639
Total output tokens: 17309
Total cost: $0.045283000000000004
